##ETL processes
This notebook develops the ETL process for each table before proceeding to complete the `etl.py` file to load the datasets.

In [1]:
!ls

ETL-processes.ipynb  Review-dataset.ipynb    data
README.md	     Star-schema-ERD.drawio  sql_queries.py


In [3]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.6 MB/s eta 0:00:00:00:01


In [4]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [5]:
conn = psycopg2.connect(host="rosie.db.elephantsql.com", database="koxdgtnq", user="koxdgtnq", password="hyik6zNrT28enwS3KAMIhRtCMlG4J2Ez")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files('data/song_data/')

In [8]:
filepath = song_files[0]

In [9]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARBEBBY1187B9B43DB,NaN,NaN,"Gainesville, FL",Tom Petty,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261,1994


1: songs Table
Extract Data for Songs Table
Select columns for song ID, title, artist ID, year, and duration
Use df.values to select just the values from the dataframe
Index to select the first (only) record in the dataframe
Convert the array to a list and set it to song_data

In [10]:
artist_id, artist_latitude, artist_location, artist_longitude, artist_name, duration, num_songs, song_id, title, year = df.values[0]
song_data = song_data = [song_id, title, artist_id, year, duration]
song_data

['A Higher Place (Album Version)', 236.17261, 1, 1994, 'Tom Petty']

Insert Record into Song Table
Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

NameError: name 'song_table_insert' is not defined